In [41]:
import json
import numpy as np

In [42]:
path = '/content/drive/MyDrive/Project Common folder/Usman_Folder/data/allData.json'

with open(path, 'r') as f:
  all_data = json.load(f)

In [43]:
len(all_data)

3017

In [44]:
all_data[0].keys()

dict_keys(['question', 'intent', 'answer'])

In [45]:
legal_list = []
def_list = []

for data in all_data:
  if data['intent']=='legal':
    legal_list.append(data)
  if data['intent']=='definition':
    def_list.append(data)

legal_list = legal_list[:900]
def_list = def_list[:900]

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [47]:
questions = []
intents = []


for data_point in legal_list:
  questions.append(data_point['question'])
  intents.append(data_point['intent'])
for data_point in def_list:
  questions.append(data_point['question'])
  intents.append(data_point['intent'])

In [48]:
print("Number of question: ", len(questions), "\nNumber of labels: ", len(intents))

Number of question:  1800 
Number of labels:  1800


In [49]:
labels = [0 if element == 'definition' else 1  for element in intents ]

In [50]:
labels[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [51]:
labels = np.array(labels)
utterances = np.array(questions)

In [26]:
utterances.shape

(1800,)

In [86]:
indices = np.arange(utterances.shape[0])
np.random.shuffle(indices)
indices

array([ 562, 1006, 1599, ..., 1227,  900, 1412])

In [87]:
utterances = utterances[indices]
labels = labels[indices]

In [88]:
train_X = utterances[:700]
train_y = labels[:700]

test_X = utterances[700:]
test_y = labels[700:]

In [89]:
train_y[:10]

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [104]:
raw_train_data =  tf.data.Dataset.from_tensor_slices((train_X,train_y))

raw_val_data = tf.data.Dataset.from_tensor_slices((test_X,test_y))

In [102]:
for text_batch, label_batch in raw_train_data.take(1):

  print(chr(text_batch.numpy()[4]))
  print(label_batch.numpy())
  # for i in range(3):
  #   print("Review", text_batch.numpy()[i])
  #   print("Label", label_batch.numpy()[i])

a
1


In [80]:
next(iter(raw_train_data))

(<tf.Tensor: shape=(), dtype=string, numpy=b'Can we use agency agreements for portfolio managers?'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [105]:
import re
import string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
  
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_data.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [76]:
for j, item in enumerate(train_text):
  print(item)
  if j == 2:
    break

tf.Tensor(b'Can we use agency agreements for portfolio managers?', shape=(), dtype=string)
tf.Tensor(b'What acts may the commissioned agent perform on behalf of the grantor?', shape=(), dtype=string)
tf.Tensor(b'What does an agents failure in conforming to his grantors specific instructions result in?', shape=(), dtype=string)


In [106]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  label =  tf.expand_dims(label, -1)
  return vectorize_layer(text), label

In [107]:
train_ds = raw_train_data.map(vectorize_text)
val_ds = raw_val_data.map(vectorize_text)
# test_ds = raw_test_ds.map(vectorize_text)

In [108]:
next(iter(train_ds))

(<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
 array([[  77,   20,    2,   69,  296, 1220,   28,    5,   46,   41,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, 

In [109]:
from tensorflow.keras import layers

embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [110]:
from tensorflow.keras import losses

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
type(labels)

numpy.ndarray

In [ ]:
test_y

array([0, 1, 1, ..., 0, 1, 0])

In [111]:
model.fit(train_ds,   epochs = 100, validation_data= train_ds, validation_steps=1)

Epoch 1/100
700/700 [==============================] - 3s 3ms/step - loss: 0.6906 - binary_accuracy: 0.5240 - val_loss: 0.7927 - val_binary_accuracy: 0.0000e+00
Epoch 2/100
700/700 [==============================] - 2s 3ms/step - loss: 0.6764 - binary_accuracy: 0.6238 - val_loss: 0.8162 - val_binary_accuracy: 0.0000e+00
Epoch 3/100
700/700 [==============================] - 2s 3ms/step - loss: 0.6384 - binary_accuracy: 0.7805 - val_loss: 0.8302 - val_binary_accuracy: 0.0000e+00
Epoch 4/100
700/700 [==============================] - 3s 4ms/step - loss: 0.5758 - binary_accuracy: 0.8331 - val_loss: 0.8368 - val_binary_accuracy: 0.0000e+00
Epoch 5/100
700/700 [==============================] - 3s 4ms/step - loss: 0.5041 - binary_accuracy: 0.8640 - val_loss: 0.8070 - val_binary_accuracy: 0.0000e+00
Epoch 6/100
700/700 [==============================] - 2s 3ms/step - loss: 0.4368 - binary_accuracy: 0.8892 - val_loss: 0.7746 - val_binary_accuracy: 0.0000e+00
Epoch 7/100
700/700 [=============

In [112]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

In [153]:
export_model.save('baseline_model_trained')

INFO:tensorflow:Assets written to: baseline_model_trained/assets


In [114]:
label_map = {0 : 'definition', 1 : 'legal'}

In [168]:
examples = [
  "Do Muslims live in America",
  "What is takaful in Islamic banking"]

pred = export_model.predict(examples)



In [169]:
answers = list(zip(list(np.where(pred < 0.5, 'def', 'legal').flatten()), list(pred.flatten())))
answers

[('legal', 0.6251091), ('def', 0.21467099)]